    # What drives the price of a car?

![](images/kurt.jpeg)

**OVERVIEW**

In this application, you will explore a dataset from Kaggle. The original dataset contained information on 3 million used cars. The provided dataset contains information on 426K cars to ensure speed of processing.  Your goal is to understand what factors make a car more or less expensive.  As a result of your analysis, you should provide clear recommendations to your client -- a used car dealership -- as to what consumers value in a used car.

### CRISP-DM Framework

<center>
    <img src = images/crisp.png width = 50%/>
</center>


To frame the task, throughout our practical applications, we will refer back to a standard process in industry for data projects called CRISP-DM.  This process provides a framework for working through a data problem.  Your first step in this application will be to read through a brief overview of CRISP-DM [here](https://mo-pcco.s3.us-east-1.amazonaws.com/BH-PCMLAI/module_11/readings_starter.zip).  After reading the overview, answer the questions below.

### Business Understanding

From a business perspective, we are tasked with identifying key drivers for used car prices.  In the CRISP-DM overview, we are asked to convert this business framing to a data problem definition.  Using a few sentences, reframe the task as a data task with the appropriate technical vocabulary.

Given that our stakeholders are seeking to understand what factors drive the price of a used car then their goal is to improve their sales and meet market demand. They have provided as a dataset of used car sales and have asked us to determine which factors make a car more or less expensive. A recent [report](https://www.statista.com/statistics/183713/value-of-us-passenger-cas-sales-and-leases-since-1990/) shows that used car sales doubles the amount of new car sales. The used car market is lucrative with high demand from consumers. Our stakeholder wants to stay ahead of the curve and understanding their customer's preferences can help lead to more sales. Used car dealerships also have to acquire cars for inventory from other entities. If our stakeholder can reasonably determine the cost of a used vehicle they can better understand their margins and when to purchase a vehicle for inventory. It is also important to understand what customers value in used cars. Knowing what drives a consumer to purchase a used vehicle can help our stakeholders avoid costly mistakes when acquiring inventory.

### Data Understanding

After considering the business understanding, we want to get familiar with our data.  Write down some steps that you would take to get to know the dataset and identify any quality issues within.  Take time to get to know the dataset and explore what information it contains and how this could be used to inform your business understanding.

In [ ]:
import plotly.express as px
import pandas as pd
import numpy as np
import plotly.io as pio

from plotly.subplots import make_subplots
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from warnings import filterwarnings

filterwarnings('ignore')

In [ ]:
# Load data
vehicles_df = pd.read_csv('data/vehicles.csv')

In [ ]:
# First we should get the basic facts of the data (describe, info, null checks, duplicate checks)
vehicles_df.info()

The id and VIN column are not important for analysis so we will definitely remove them.

In [ ]:
vehicles_df.describe()

In [ ]:
vehicles_df.describe(include='object')

In [ ]:
print(f'Row count: {vehicles_df.shape[0]}, Duplicate count: {vehicles_df.shape[0] - vehicles_df.drop_duplicates().shape[0]}')

In [ ]:
vehicles_df.isna().mean().round(2)

Luckily, we do not have duplicates. However, we are missing values, specifically the condition and cylinders attributes are missing in over 40% of rows. Color is missing in 31% of rows. Size is missing in 72% of rows. Drive is missing in 31% of rows. I know that these attributes are important in determining the price of a car from prior analysis. The amount of missing data is concerning and the best course of action may be to impute them. That may mislead our model and we should be wary of that. However, dropping over 40% of the data isn't something we should do. One way to impute this data would be to find a finding an identical year and model then we would select the most seen value for that respective column; we can leverage scipy's `SimpleImputer`. This can help us deduce the missing attributes.

I will create an imputed dataset and run my analysis on both of them.

In [ ]:
# predefine to avoid extra computation
groupby_model_year = vehicles_df.groupby(['model', 'year'])
# This somewhat mimics SimpleImputer. If we wanted to get fancy we would use KNN and leverage sales price in imputation
def impute_field_by_model_year(row, field_to_update):
    # ensure fields are not na and some non na value exists
    try:
        # take the most common occurrences for model and year
        group = groupby_model_year.get_group((row['model'], row['year']))
        non_na_values = group[field_to_update].dropna()
        if not non_na_values.empty:
            return non_na_values.mode().iloc[0]
    except KeyError:
        return np.nan
    return np.nan

# copy vehicles df and impute columns missing values
imputed_vehicle_df = vehicles_df.copy(deep=True)
fields_to_impute = ['condition', 'cylinders', 'drive', 'size', 'type', 'paint_color']
for field in fields_to_impute:
    na_filter = imputed_vehicle_df[field].isna()
    imputed_vehicle_df.loc[na_filter, field] = imputed_vehicle_df[na_filter].apply(lambda row: impute_field_by_model_year(row, field), axis=1)


In [ ]:
imputed_vehicle_df.isna().mean().round(2)


The imputed vehicle dataframe is looking a lot better.
<table>
<tr>
<th>column</th>
<th>old missing value percent</th>
<th>new missing value percent</th>
</tr>
<tr>
<td>condition</td>
<td>0.41</td>
<td>0.09</td>
</tr>
<tr>
<td>condition</td>
<td>0.42</td>
<td>0.14</td>
</tr>
<td>drive</td>
<td>0.31</td>
<td>0.10</td>
</tr>
</tr>
<td>size</td>
<td>0.72</td>
<td>0.30</td>
</tr>
</tr>
<td>type</td>
<td>0.22</td>
<td>0.05</td>
</tr>
</tr>
<td>paint_color</td>
<td>0.31</td>
<td>0.08</td>
</tr>
</table?


Let's start by checking a histogram of sales prices

In [ ]:
fig = px.box(vehicles_df, x='price', title='Price attribute has unrealistic values',
                   labels={'price': 'Price', 'count': 'Count'})
fig.show()
fig.write_image('images/box_amount.png')

The above is not realistic. We are seeing vehicles selling for billions. I will remove outliers using iqr.

In [ ]:
q1 = vehicles_df['price'].quantile(0.25)
q3 = vehicles_df['price'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr)
upper_bound = q3 + (1.5 * iqr)

vehicles_df_no_outlier = vehicles_df.query(f'price >= {lower_bound} and price <= {upper_bound}')
imputed_vehicle_df = imputed_vehicle_df.query(f'price >= {lower_bound} and price <= {upper_bound}')

In [ ]:
fig = px.box(vehicles_df_no_outlier, x='price', title='Majority of vehicles fall between &#36;5,799 - &#36;25,590',
                   labels={'price': 'Price', 'count': 'Count'})
fig.show()
fig.write_image('images/box_amount_outliers.png')

x### Data Preparation

After our initial exploration and fine-tuning of the business understanding, it is time to construct our final dataset prior to modeling.  Here, we want to make sure to handle any integrity issues and cleaning, the engineering of new features, any transformations that we believe should happen (scaling, logarithms, normalization, etc.), and general preparation for modeling with `sklearn`. 

### Modeling

With your (almost?) final dataset in hand, it is now time to build some models.  Here, you should build a number of different regression models with the price as the target.  In building your models, you should explore different parameters and be sure to cross-validate your findings.

### Evaluation

With some modeling accomplished, we aim to reflect on what we identify as a high-quality model and what we are able to learn from this.  We should review our business objective and explore how well we can provide meaningful insight into drivers of used car prices.  Your goal now is to distill your findings and determine whether the earlier phases need revisitation and adjustment or if you have information of value to bring back to your client.

### Deployment

Now that we've settled on our models and findings, it is time to deliver the information to the client.  You should organize your work as a basic report that details your primary findings.  Keep in mind that your audience is a group of used car dealers interested in fine-tuning their inventory.